In [ ]:
!git clone --recurse-submodules https://github.com/gicait/pLitter.git
%cd pLitter
!pip install -e .

In [ ]:
!mkdir ./data
!wget -O ./data/pLitterStreet_Sample_DATA.zip https://github.com/gicait/pLitter/releases/download/v0.0.0-street/pLitterStreet_Sample_DATA.zip
!unzip ./data/pLitterStreet_Sample_DATA.zip -d ./data/.
# !rm -rf './data/pLitterStreet_Sample_DATA.zip'

In [ ]:
from plitter import spatial
from plitter.detector import detector, draw_boxes_and_count_on_image
import time
import cv2

model = detector('cctv')
# Open a sample riverine video or image
cap = cv2.VideoCapture('../path_to_riverine_video.mp4')  # Or use cv2.imread() for an image

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run detection on the frame
    preds = model(frame, size=1280)
    # Extract prediction details
    boxes = preds.xyxy[0][:, :4].tolist()
    scores = preds.xyxy[0][:, 4].tolist()
    class_ids = preds.xyxy[0][:, 5].tolist()
    classes = [model.names[int(i)] for i in class_ids]

    # Annotate the frame with boxes and the count overlay
    annotated_frame = draw_boxes_and_count_on_image(frame, boxes, classes, class_ids, scores)

    # Display the result
    cv2.imshow('Annotated Frame', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
route = spatial.pTrack('./data')

start = time.time()
route.load()
route.detect(model, skip_frames=4)
end = time.time()

print("Time taken: ", end - start, " seconds")

route.export('./data/sample.geojson')

In [ ]:
from plitter import spatial
from plitter.detector import detector, tracker
import time

model = detector('street')
tracker = tracker('strongsort')

route = spatial.pTrack('./data')

start = time.time()
route.load()
route.detect(model, tracker, skip_frames=0)
end = time.time()

print("Time taken: ", end - start, " seconds")

route.export('./data/sample.geojson')

In [ ]:
# usb camera and using tracker

# !python3 plitter/stationCam.py --sensor-id 0 --dest data
# !python3 plitter/stationCam.py --sensor-id 0 --dest data --csi-cam
# !python3 plitter/stationCam.py --sensor-id 0 --dest data --use-tracker
# !python3 plitter/stationCam.py --sensor-id 0 --dest data --use-tracker --save